In [2]:
from os import path
import pandas as pd
import radd
from radd import build
%matplotlib inline

In [10]:
_homedir = path.expanduser('~')
# change this to wherever you have your data stored
coxon_data_path = path.join(_homedir, "Dropbox/Projects/Coxon/coxon_behav_data.csv")
# read data into pandas DataFrame (http://pandas.pydata.org/)
coxon_data = pd.read_csv(coxon_data_path)

# initiate dependent process model with dynamic gain (kind = 'xdpm')
# (change kind='dpm' for model with no dynamic gain signal)
# drift-rate (v) depends on Baseline and Caution levels of 'Cond' variable
dpm = build.Model(kind='xdpm', data=coxon_data, depends_on={'v':'Cond'}, fit_on='average')

# initiate independent race model with dynamic gain (kind = 'xirace')
# irm = build.Model(kind='xirace', data=coxon_data, depends_on={'v':'Cond'}, fit_on='average')

## Animation of Dependent Process Model

In [59]:
# Initial state of Stop process (red) depends on current strength of Go activation (green)
# Assumes Stop signal efficacy at later SSDs diminishes as the state of the Go process 
# approaches the execution threshold (upper bound). pink lines denote t=SSD, blue is trial deadline
radd.load_dpm_animation()

### columns in model's observed dataframe (model.observedDF)
* **idx**: subject ID
* **Cond**: Baseline(bsl)/Caution(pnl) (could be any experimental condition of interest) 
* **Acc**: Accuracy on "go" trials
* **sacc**: Mean accuracy on "stop" trials (mean condition SSD used during simulations)
* **c10 - c90**: 10th - 90th RT quantiles for correct responses
* **e10 - e90**: 10th - 90th RT quantiles for error responses

In [60]:
dpm.observedDF.head()

,idx,Cond,acc,sacc,c10,c30,c50,c70,c90,e10,e30,e50,e70,e90
0,1,pro,1.0,0.52500,0.76784,0.79500,0.8095,0.82400,0.84916,0.76066,0.77700,0.7895,0.80356,0.82402
1,1,reac,1.0,0.51613,0.76300,0.78300,0.7960,0.81634,0.84000,0.75642,0.76746,0.7850,0.79854,0.81216
2,2,pro,1.0,0.48750,0.78442,0.80200,0.8150,0.83500,0.85158,0.76300,0.78600,0.7970,0.81100,0.82200
3,2,reac,1.0,0.46774,0.78200,0.79800,0.8070,0.81914,0.83600,0.76500,0.77900,0.7890,0.80722,0.82800
4,3,pro,1.0,0.50000,0.77282,0.79366,0.8130,0.83500,0.86200,0.75310,0.78300,0.7945,0.81754,0.84812


## Optimize DPM to the Average Data

In [64]:
# Set basinhopping step-size to .1 (see HopStep class in radd.fit 
# and get_stepsize_scalars function in radd.tools.theta). 
# Sample 5000 possible parameters sets, perform global optimization on best 5
# then local optimization on lowest (best) 1 of 5 
dpm.set_basinparams(stepsize=.1, nsamples=500, ninits=5, tol=1e-20, nsuccess=20)
dpm.set_fitparams(tol=1e-30)

In [65]:
# fit DPM using combination of global (basinhopping + TNC)
# and local gradient optimization (Nelder-Mead Simplex)
# set progress=True to track global minimum across all (5) intit params
# as well as the fmin (basin) of the current global optimization process
# ("progress" output is erased after fits are run... you'll see it when you run this cell)
#dpm = build.Model(kind='xdpm', data=coxon_data, depends_on={'v':'Cond'}, fit_on='average', weighted=True)
dpm.set_fitparams(ntrials=20000, tol=1e-20)
dpm.optimize(plot_fits=True, progress=True)

In [68]:
dpm.finfo

v_pro       0.95757
v_reac      0.98610
a           0.36219
xb          0.56105
ssv        -0.89071
tr          0.45903
cnvrg       1.00000
nfev      126.00000
nvary       2.00000
chi         0.00177
ndata      24.00000
df         22.00000
rchi        0.00008
logp     -226.29456
AIC      -222.29456
BIC      -222.42336
idx         1.00000
dtype: float64

## Some Further Explanation...

* All models are initially fit to a flat data vector, weighted by an equal length array of weights estimated as the inverse of the measured uncertainty in each observed data point (flat meaning single-dimensional vector, collapsing across conditions)

* If the user supplies a **depends_on** dictionary when instantiating the model (see above), then all parameters from the initial (flat) fit are held constant except for the parameter in **depends_on.keys()** which is free to vary across levels of the condition in **depends_on.values()**. 
    * For instance, to fit a model with drift-rate free to vary across levels of 'Cond' (column in the data)            
        * depends_on = {'v': 'Cond'} 
   
* For **model** with **fit_on** = 'average' and **depends_on** = {**param** : **condition**}

|model information | method used to calculate | how to access|
|--|--|--|
| flat data | **model**.observedDF.mean() | **model**.observed_flat |
| flat weights | **model**.wtsDF.mean() | **model**.flat_wts |
| conditional data | **model**.observedDF.groupby(**condition**).mean()| **model**.observed |
| conditional weights | **model**.wtsDF.groupby(**condition**).mean() |  **model**.cond_wts |

## Troubleshooting Ugly Fits

* fit to individual subjects
    * model = build.Model(data=data, ..., **fit_on**=**'subjects'**)
* other models (currently only dpm and irace), 
    * the example above uses the dpm with dynamic rate bias (**kind**=**'xdpm'**)
    * to implement the independent race model with the same drift-rate dependency on Cond as above
        * model = build.Model(data=data, **kind**=**xirace'**, **depends_on**={**'v'**: **'Cond'**})
    * implement without dynamic bias (same "kind" without the 'x' )
        * model = build.Model(data=data, **kind**=**irace'**, **depends_on**={**'v'**: **'Cond'**})
        
* other dependencies, maybe subjects change their boundary height or go onset time across conditions
    * model with boundary free across conditions:
        * model = build.Model(data=data, ... , **depends_on**={**'a'**: **'Cond'**})
    * model with onset free across conditions:
        * model = build.Model(data=data, ... , **depends_on**={**'tr'**: **'Cond'**})
        
* increasing size of the parameters search and setting more conservative convergence criteria 
     * model.set_basinparams(niter_success=50, tol=1e6, nrand_inits=10, nrand_samples=10000) 
     * model.set_fitparams(maxfev=5000, tol=1e-6)
     * check out the wts vectors for extreme vals
         * try re-running the fits with an unweighted model (all wts = 1) 
             * m = build.Model(data=data, ... weighted=False)
     * sometimes error rts are particularly troublesome, sometimes un-shootably so...


### double check the mean ssd values for pro and reac conditions look right?
(important as these were used to simulate the model predictions entered into the cost_fx)

   * pro:  0.63064
   * reac: 0.62547


## Examine fits

In [8]:
# the fit summary (goodness of fit measures, etc.) 
dpm.fitDF

idx,average
a,0.38918
ssv,-0.8348
tr,0.45325
xb,1.6683
v_pro,0.86255
v_reac,0.88338
nfev,41
nvary,2
df,22
chi,0.0011883


In [9]:
# model predictions
# to save as csv file: dpm.yhatDF.to_csv("path_where_you_want_to_save_to", index=False)
# to extract values as numpy ndarray: dpm.yhatDF.loc[:, 'acc':].values
dpm.yhatDF

,idx,Cond,acc,sacc,c10,c30,c50,c70,c90,e10,e30,e50,e70,e90
0,average,pro,0.9996,0.4812,0.77825,0.80325,0.81825,0.84325,0.87825,0.76325,0.78825,0.79825,0.81325,0.83825
1,average,reac,0.9998,0.4935,0.77325,0.79825,0.81825,0.83825,0.86825,0.76325,0.77825,0.79825,0.81325,0.82825


In [10]:
# best-fit parameter estimates also stored in popt dictionary
dpm.popt

{'a': 0.38918227126778043,
 'ssv': -0.83480446499351579,
 'tr': 0.4532545317561319,
 'v': array([ 0.86255,  0.88338]),
 'v_pro': 0.86254648637513887,
 'v_reac': 0.88338189769825337,
 'xb': 1.6683142202828718}

In [1]:
import radd
radd.style_notebook()

Notebook Theme: Grade3
more at github.com/dunovank/jupyter-themes
